In [1]:
"""
@author: albertigno

"""

from MyDataset import *
import torch, time, os
import torch.nn as nn
import torch.nn.functional as F
#import networkx as nx
import matplotlib.pyplot as plt
#from matplotlib.gridspec import GridSpec

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ('Running on: {}'.format(device))

Running on: cuda:0


In [2]:
thresh = 0.3
batch_size = 256 # default 256
learning_rate = 1e-4 # default 1e-4
time_window = 50 # shd 50, nmnist 25-30
dataset_path = r'./../../datasets'

In [3]:
train_path = dataset_path+'/shd_digits/shd_train.h5'
test_path = dataset_path+'/shd_digits/shd_test.h5'
# load datasets
print("loading test set...")
test_dataset = MyDataset(test_path, 'hd_digits', time_window, device)
print("loading training set...")
train_dataset = MyDataset(train_path, 'hd_digits', time_window, device)
print("loading data with pytorch")
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False, drop_last=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True, drop_last=True)

loading test set...
num sample: 2264
torch.Size([2264, 50, 700]) torch.Size([2264, 20])
loading training set...
num sample: 8156
torch.Size([8156, 50, 700]) torch.Size([8156, 20])
loading data with pytorch


In [46]:
from snn_models import *
%load_ext autoreload
%autoreload 1
%aimport snn_models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
num_hidden = 512
#tau_m = 'adp'
tau_m = 0.8305
#snn = RSNN_delay(d='shd', num_hidden=128, thresh=0.3, decay=0.3, batch_size=batch_size, win=50, device=device)
snn = RSNN_f('shd', num_hidden=num_hidden, thresh=0.3, tau_m=tau_m, vreset=0.0, batch_size=batch_size, win=time_window, device=device)
snn.to(device)

RSNN_f(
  (fc_ih): Linear(in_features=700, out_features=512, bias=False)
  (fc_hh): Linear(in_features=512, out_features=512, bias=False)
  (fc_ho): Linear(in_features=512, out_features=20, bias=False)
)

In [6]:
# training configuration

num_epochs = 100
modelname = 'shd_rnn_{}_{}.t7'.format(snn.num_hidden, tau_m)
num_samples = train_dataset.images.size()[0]

# super pythonic way to extract the parameters that will have 'normal' learning rate
base_params = [getattr(snn,name.split('.')[0]).weight for name, _ in snn.state_dict().items() if name[0]=='f']

# setting different learning rate for tau_m, if neeeded
if tau_m=='adp':
    print('tau_m_h ')
    optimizer = torch.optim.Adam([
        {'params': base_params},
        {'params': snn.tau_m_h, 'lr': learning_rate * 10.0},
        {'params': snn.tau_m_o, 'lr': learning_rate * 10.0}],
        lr=learning_rate)
else:    
    optimizer = torch.optim.Adam([
        {'params': base_params},
        {'params': snn.delay_index, 'lr': learning_rate * 10000.0}],        
        lr=learning_rate)
    
act_fun = ActFun.apply
print(modelname)

shd_rnn_512_0.8305.t7


In [7]:
for parameter in snn.parameters():
    print(parameter)

Parameter containing:
tensor([0.8305], device='cuda:0')
Parameter containing:
tensor([1.2999e-01, 1.1847e+00, 3.9112e+00, 6.0608e+00, 2.4888e+00, 2.9835e+00,
        5.7574e+00, 3.8007e+00, 1.4161e+00, 7.6868e+00, 6.8888e+00, 6.8583e+00,
        4.6502e+00, 7.9625e+00, 3.0535e+00, 7.9934e+00, 5.7683e+00, 2.3399e+00,
        2.0489e+00, 3.4227e-01, 2.6783e+00, 4.2200e+00, 3.5828e+00, 2.0558e+00,
        1.6456e-01, 5.7457e+00, 5.4135e+00, 2.6509e+00, 4.9221e+00, 6.7725e+00,
        3.8806e+00, 2.2331e+00, 4.2570e+00, 6.7304e+00, 1.8111e+00, 1.5787e+00,
        6.9351e+00, 3.6239e+00, 4.5638e+00, 7.8124e+00, 1.9003e+00, 5.3211e+00,
        6.2198e+00, 5.0019e+00, 2.2079e-01, 3.5807e+00, 6.3870e+00, 3.3396e+00,
        3.6976e+00, 7.4276e+00, 5.9437e+00, 5.7276e+00, 5.9185e+00, 6.3804e+00,
        3.8075e+00, 4.3315e+00, 6.4329e+00, 2.9846e+00, 7.1206e+00, 3.0366e+00,
        5.6227e+00, 5.8639e+00, 5.2018e+00, 5.2756e+00, 5.8377e+00, 5.6412e+00,
        5.1126e+00, 2.5268e+00, 4.2954e+00

In [8]:
before= snn.delay_index.data

In [9]:
# training loop
taus_m = []
for epoch in range(num_epochs):
    print('Epoch [%d/%d]'  % (epoch + 1, num_epochs))
    start_time = time.time()
    snn.train_step(train_loader, optimizer=optimizer, criterion=nn.MSELoss(), num_samples = num_samples)
    t = time.time() - start_time
    print('Time elasped:', time.time() - start_time)
    
    # update learning rate
    optimizer = snn.lr_scheduler(optimizer, lr_decay_epoch=1)
    
    # weight and decay recording
    # taus_m.append((snn.tau_m_h.data.detach().clone(), snn.tau_m_o.data.detach().clone()))
    
    if (epoch+1) % 5 ==0:
        snn.test(test_loader, criterion=nn.MSELoss())
        #snn.save_model(modelname)   
                
with open('training_log', 'a') as logs:
    logs.write("\nFinished training {} epochs for {}, batch_size {}, time_per_epoch {} s".format(num_epochs, modelname, batch_size, t))       

Epoch [1/100]
tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

...

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

tensor([0, 1, 3, 6, 2, 2, 5, 3, 1, 7, 6, 6, 4, 7, 3, 7, 5, 2, 2, 0, 2, 4, 3, 2,
        0, 5, 5, 2, 4, 6, 3, 2, 4, 6, 1, 1, 6, 3, 4, 7, 1, 5, 6, 5, 0, 3, 6, 3,
        3, 7, 5, 5, 5, 6, 3, 4, 6, 2, 7, 3, 5, 5, 5, 5, 5, 5, 5, 2, 4, 3, 4, 2,
        6, 6, 4, 4, 7, 7, 3, 1, 5, 5, 5, 4, 7, 0, 1, 2, 0, 7, 3, 2, 1, 6, 1, 5,
        5, 1, 6, 0, 0, 5, 3, 3, 2, 2, 5, 1, 1, 7, 5, 6, 1, 2, 0, 1, 0, 3, 4, 6,
        4, 7, 3, 6, 0, 5, 6, 7, 7, 4, 7, 6, 6, 1, 3, 3, 2, 6, 4, 1, 6, 2, 5, 4,
        0, 3, 2, 2, 3, 0, 2, 4, 7, 1, 7, 3, 6, 5, 0, 7, 6, 2, 7, 5, 6, 7, 5, 7,
        1, 0, 3, 1, 4, 3, 0, 1, 1, 3, 0, 6, 1, 3, 7, 3, 4, 7, 5, 7, 2, 5, 5, 1,
        0, 7, 0, 5, 7, 3, 5, 2, 5, 6, 5, 0, 2, 1, 1, 1, 5, 1, 0, 2, 5, 0, 2, 7,
        2, 7, 0, 1, 0, 5, 3, 1, 0, 1, 4, 6, 6, 1, 3, 2, 5, 6, 7, 4, 1, 4, 2, 2,
        4, 2, 7, 3, 1, 5, 6, 2, 5, 3, 4, 2, 3, 1, 3, 2, 2, 3, 0, 2, 2, 5, 5, 7,
        5, 4, 3, 2, 3, 6, 7, 6, 7, 2, 4, 0, 7, 6, 2, 5, 7, 0, 3, 6, 6, 3, 2, 5,
        5, 2, 4, 3, 2, 0, 3, 4, 4, 7, 6,

KeyboardInterrupt: 

In [10]:
after = snn.delay_index.data

In [11]:
before==after

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

##### Index training failed